### Introduction

Welcome to Priceless Wheels! In this project, our goal is to build a model that can accurately predict the price of a used vehicle based on various factors such as make, model, year, mileage, and condition. The automobile industry is one of the largest and most competitive industries in the world, with millions of vehicles being sold each year. The price of a vehicle can have a significant impact on a consumer's purchasing decision and it is important for both buyers and sellers to have an understanding of the market value of a vehicle. By using machine learning algorithms and data analysis, we aim to provide a reliable and robust model that can assist in determining the fair market value of a vehicle. Join us on this exciting journey as we delve into the world of vehicle price prediction.

### About the data

This data is scrapped from https://www.cardekho.com/. This data is meant for research and academic purposes only and is **not meant for commercial use**. This dataset contains about 38000 (thirty eight thousand) used cars listed in CarDekho in India. Download the data from https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv.

---

#### Importing the data and libraries

Let's start by importing the necessary libraries and the data.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import regex as re # for regex matching
import matplotlib.pyplot as plt # 
import seaborn as sns # seaborn for nice looking plots
sns.set() # setting seaborn default for plots

# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/used-cars-dataset-cardekho/car_details_merged_cleaned.csv
/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv
/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv


Next, we will import the data and preview the first 5 rows

- Note: The ```usedCarSkuId``` is a unique identifier for each car. We would be using this column as the index of the dataframe
- Note: Since we are using github lfs for storing the CSV files, the link mentioned in the notebook will expire in some time. Please visit this [link](https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv) to get the data

In [2]:
# The data can be found on kaggle link: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

file_path = "/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv"
df = pd.read_csv(file_path, index_col="usedCarSkuId")

# sanity check
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (25,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN


The data has imported with some mixed data types. We will need to convert the data types as we explore the data.

In [3]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 37814 rows and 139 columns


As we can see, there are 37,814 rows and 139 columns in the dataset.

Let's look at the data types of the columns.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37814 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Columns: 139 entries, position to Ground Clearance Unladen
dtypes: bool(3), float64(10), int64(28), object(98)
memory usage: 39.6+ MB


Let's take a look at the feature dictionary provided with the dataset.

In [5]:
feature_dictionary = pd.read_csv("/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv")
feature_dictionary

,Feature,Type,Sample,Missing Values,Unique Values,Description
0,position,int64,1,0,20,Position of the car in the list
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
...,...,...,...,...,...,...
135,experiment,object,control,0,1,control
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


Now comes the difficult task. Go through the feature dictionary and remove the unwanted columns - features which might be repeated but not totally apparent. Note down the columns which makes sense to keep and are not repeating and delete the rest.

In [6]:
columns_to_keep = [
    "loc",
    "myear",
    "bt",
    "tt",
    "ft",
    "km",
    "ip",
    "images",
    "imgCount",
    "threesixty",
    "dvn",
    "oem",
    "model",
    "variantName",
    "city_x",
    "pu",
    "discountValue",
    "utype",
    "carType", 
    "top_features",
    "comfort_features",
    "interior_features",
    "exterior_features",
    "safety_features",
    "Color",
    "Engine Type",
    "Max Power",
    "Max Torque",
    "No of Cylinder",
    "Values per Cylinder",
    "Value Configuration",
    "BoreX Stroke",
    "Turbo Charger",
    "Super Charger",
    "Length",
    "Width",
    "Height",
    "Wheel Base",
    "Front Tread",
    "Rear Tread",
    "Kerb Weight",
    "Gross Weight",
    "Gear Box",
    "Drive Type",
    "Seating Capacity",
    "Steering Type",
    "Turning Radius",
    "Front Brake Type",
    "Rear Brake Type",
    "Top Speed",
    "Acceleration",
    "Tyre Type",
    "No Door Numbers",
    "Cargo Volumn",
    "model_type_new",
    "state",
    "owner_type",
    "exterior_color",
    "Fuel Suppy System",
    "Compression Ratio",
    "Alloy Wheel Size",
    "Ground Clearance Unladen",
]

df.drop([x for x in df.columns if x not in columns_to_keep], axis=1, inplace=True)
print(f"After dropping some unnecessary columns, the dataset now has {df.shape[1]} columns. These columns are hand picked and will be be further analyzed.")

After dropping some unnecessary columns, the dataset now has 62 columns. These columns are hand picked and will be be further analyzed.


---

### Data cleaning and wrangling

* Dropping duplicate rows
* Fixing the values and data types of the columns
* Checking for multicollinearity and determining how to handle it
* Dropping irrelevant columns for the model
* Saving the cleaned data for the next part of the project

---

### Duplicates

We know that there should be no duplicates in the data. They are checked for and removed at the time of data collection. *(Trust me, I collected it myself 😉 )*

In [7]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 1.


Somehow, one duplicate row sneaked in (that's awkward 🫣)! Let's check the rows.

In [8]:
# setting the duplicated index
duplicate_index = df.duplicated(keep=False)

# calling the duplicated index in a dataframe
df.loc[duplicate_index, :].sort_index()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
aa39e640-6183-4379-a517-9f5b2458b2a5,NaN,2014,Hatchback,Manual,Diesel,"70,000",0,[{'img': ''}],0,False,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN
ce8b30d8-c438-4a2f-bce1-5c1887a95495,NaN,2014,Hatchback,Manual,Diesel,"70,000",0,[{'img': ''}],0,False,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN


As we can see, the rows are exactly the same. This means we can safely drop the row. Since the we indexed by the usedCarSkuId, we need to drop one of them by their index.

In [9]:
# making a copy with the duplicated rows dropped
df2 = df.drop_duplicates().copy()

# checking for duplicate rows in the new dataframe
dup = df2.duplicated().sum()

print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [10]:
# sanity check
df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,180-liters,used,uttar pradesh,Silver,first,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,235-litres,used,maharashtra,Grey,first,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,4.000,400-litres,used,delhi,Silver,second,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,[{'img': ''}],0,False,...,4.000,NaN,used,delhi,Silver,second,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,5.000,NaN,used,maharashtra,White,first,NaN,12.0+-.03,NaN,NaN


Great! Now that we have removed the duplicate rows, let's look at missing values

---

### Fixing the data types

Let's look at the data types of the columns and in the process we can check if we need to clean any them to make them more useful, such as converting them to a numeric type.

*When we do feature engineering we will further modify these columns and might also drop a few of them*

We already know about the current data types of the features from the ```feature_dictionary```.

In [11]:
# Filter the dataframe to only include the columns_to_keep
new_feature_dict = feature_dictionary.copy()

# Drop the rows which Feature is not in the columns_to_keep list
row_index = [i for i, x in enumerate(new_feature_dict['Feature']) if x not in columns_to_keep]
new_feature_dict.drop(row_index, inplace=True)

new_feature_dict

,Feature,Type,Sample,Missing Values,Unique Values,Description
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
5,ft,object,CNG,0,5,Fuel type of the car
...,...,...,...,...,...,...
130,owner_type,object,first,0,6,"Owner type of the car (first, second, etc.)"
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


We know that if a column has the ```object``` type, it means it could have mixed types as well as strings. Let's try to convert them to a numerical type wherver we encouter something like Length or '12mm'.

In [12]:
# A helper function to print all the types in a column for a dataframe
def print_samples_from_every_type(df: pd.DataFrame, col: str):
    # Get all the types in the column.
    types = df[col].apply(type).unique()
    for t in types:
      # Print 10 samples from each type.
        print(f"Samples for type {t}: ")
        print(df[df[col].apply(type) == t].sample(10)[col])
        print()

In [13]:
# Make a copy of the original dataframe so that we can roll back to it anytime.
cars_df = df2.copy()

---

Let's list down all the columns which need the standard string treatment: 
1. Convert column to be a string type
2. Strip the whitespace 
3. Make it lowercase

*Here we are also give some of the other columns (that we wish to convert later) the standard treatment (it will help us later)*

In [14]:
columns_str_to_lower = [
    'loc',
    'bt',
    'ft',
    'tt',
    'images',
    'dvn',
    'oem',
    'model',
    'variantName',
    'city_x',
    # 'pu': 3,70,000 -> 370000
    'utype',
    'carType',
    'top_features',
    'comfort_features',
    'interior_features',
    'exterior_features',
    'safety_features',
    'Color',
    'Engine Type',
    'Max Power', # Max Power : 33.54bhp@4000 rpm -> 2 columns 33.54, 4000
    'Max Torque', # Max Torque : 40.2Nm@3500 rpm -> 2 columns 40.2, 3500
    # No of Cylinder : float -> int,
    # Values per Cylinder : float -> int, !! Fix the name -> Valve per Cylinder
    'Value Configuration', # !! Fix the name -> Valve Configuration
    'BoreX Stroke', # BoreX Stroke : 69 x 72 mm -> Bore: 69, Stroke: 72
    'Turbo Charger', # Convert to boolean
    'Super Charger', # Convert to boolean
    'Length', # Length : 3599mm -> 3599
    'Width', # Width : 1495mm -> 1495
    'Height', # Height : 1700mm -> 1700
    'Wheel Base', # Wheel Base : 2400mm -> 2400
    'Front Tread', # Front Tread : 1295mm -> 1295
    'Rear Tread', # Rear Tread : 1295mm -> 1295
    'Kerb Weight', # Kerb Weight : 960kg -> 960
    'Gross Weight', # Gross Weight : 1350kg -> 1350
    'Gear Box', # might need some additional cleaning
    'Drive Type', # might need some additional cleaning
    # Seating Capacity : float -> int
    'Steering Type', # might need some additional cleaning
    'Turning Radius', # Turning Radius : 4.6 metres -> 4.6
    'Front Brake Type', # might need some additional cleaning
    'Rear Brake Type', # might need some additional cleaning
    'Top Speed', # Top Speed : 137 kmph -> 137
    'Acceleration', # Acceleration : 13.5 seconds -> 13.5
    'Tyre Type', # might need some additional cleaning
    # No Door Numbers : float -> int !! fix name
    'Cargo Volumn', # 'Cargo Volumn' : 300 litres -> 300, !! fix name
    'model_type_new', # !! fix name
    'state',
    'exterior_color',
    'owner_type', # might need some additional cleaning
    'Fuel Suppy System', # might need some additional cleaning
    # Compression Ratio : 10.0:1 -> 10.0
    'Alloy Wheel Size', # Alloy Wheel Size : convert to float
    'Ground Clearance Unladen', # Ground Clearance Unladen : 170mm -> 170
]

In [15]:
# Apply the changes
temp_arr = []

for col in columns_str_to_lower:
    cars_df[col] = cars_df[col].astype(str).str.strip().str.lower()
    
    # Print a nice sample
    while (True):
        found = 0
        vals = [v for v in cars_df[col].sample(100)]
        for val in vals:
            if val is not None:
                found = 1
                vals = val
                break
        
        if found == 1:
            temp_arr.append({ 
                "Feature" : col, 
                "Type": f'{cars_df[col].dtype}', 
                "Unique": cars_df[col].nunique(), 
                "Sample": vals
            })
            break

pd.DataFrame(temp_arr)

,Feature,Type,Unique,Sample
0,loc,object,398,nan
1,bt,object,12,hatchback
2,ft,object,5,petrol
3,tt,object,2,manual
4,images,object,37135,[{'img': 'https://images10.gaadi.com/usedcar_i...
5,dvn,object,4128,chevrolet cruze ltz at
6,oem,object,46,mahindra
7,model,object,382,honda city
8,variantName,object,3430,vxi
9,city_x,object,617,new delhi


There is an additional issues here that needs to be addressed: 
Some of the columns have **more unique values** than are expected. For eg: `Turbo Charger` should either be a yes or a no or a NaN, beter are `5` unique values. The following are the columns suspected to have a higher than expected cardinality due to some quality issues: 
1. `Value Configuration`
2. `Turbo Charger`
3. `Gear Box`
4. `Drive Type`
5. `Steering Type`
6. `Front Brake Type`
7. `Rear Brake Type`
8. `Tyre Type`


In [16]:
# Make a list of such columns which seems to have a higher than expected cardinality
columns_unexpected_cardinality = [
    'Value Configuration',
    'Turbo Charger',
    'Gear Box',
    'Drive Type',
    'Steering Type',
    'Front Brake Type',
    'Rear Brake Type',
    'Tyre Type',
]

---

##### `Value Configuration`

In [17]:
cars_df[columns_unexpected_cardinality[0]].value_counts()

dohc                    23502
nan                      7831
sohc                     6200
undefined                 121
idsi                       66
dohc with vis              31
dohc with vgt              20
16 modules 48 cells        11
16-valve dohc layout        9
dohc with tis               9
vtec                        6
mpfi                        4
ohv / pushrod               3
Name: Value Configuration, dtype: int64

We have identified the following issues:
1. As we can see, the majority of it is `dohc`. There are a couple of variants of `dohc` like `dohc vis` but they are so few that it would be practical to replace all the columns containing any variant `dohc` to simply `dohc`
2. There are some `undefined` values which should be `NaN`
3. `mpfi` is a fuel injection system, so replace it with `NaN`
4. `vtec` engines can be `dohc` and `sohc` as well (`vtec` is an engine type developed by Honda in the 1960s). It would be best to replace this type by `NaN`, postpone handling it until null values handling (and there are only `6` rows, so it should be fine)

In [18]:
# Replace all the variants of `dohc` to simply `dohc`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vis', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vgt', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('16-valve dohc layout', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with tis', 'dohc')

# Replace `undefined`, `mpfi`, `vtec` with `NaN`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('undefined', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('mpfi', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('vtec', 'nan')

cars_df['Value Configuration'].value_counts()

dohc                   23571
nan                     7962
sohc                    6200
idsi                      66
16 modules 48 cells       11
ohv / pushrod              3
Name: Value Configuration, dtype: int64

---

##### `Turbo Charger`

In [19]:
cars_df[columns_unexpected_cardinality[1]].value_counts()

no       20247
yes      15262
nan       2176
twin       125
turbo        3
Name: Turbo Charger, dtype: int64

Well, cleaning this column should be quite straightforward.
1. `twin` and `turbo` should be replaced by `yes`

In [20]:
# Replace `twin` and `turbo` with `yes`
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('twin', 'yes')
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('turbo', 'yes')

cars_df['Turbo Charger'].value_counts()

no     20247
yes    15390
nan     2176
Name: Turbo Charger, dtype: int64

--- 

##### `Gear Box`

In [21]:
cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed                  22840
6 speed                   5876
5-speed                   1457
7 speed                   1256
6-speed                   1199
                         ...  
six speed manual             1
7g-dct                       1
amg speedshift dct 8g        1
amg 7-speed dct              1
ecvt                         1
Name: Gear Box, Length: 106, dtype: int64

Here the cardinality is high. And as is evident, several categories can be lumped together into a single one. Let's define a mapping to clearly indeicate which data should be lumped together.

In [22]:
gear_box_mapping = {}
gear_box_mapping['1 speed'] = [
	'single speed', 
	'single speed automatic', 
	'single speed reduction gear', 
	'single-speed transmission', 
]
gear_box_mapping['4 speed'] = [
	'4 speed', 
	'4-speed', 
]
gear_box_mapping['5 speed'] = [
	'5', 
	'5 - speed', 
	'5 gears', 
	'5 manual', 
	'5 speed', 
	'5 speed at+ paddle shifters', 
	'5 speed cvt', 
	'5 speed forward, 1 reverse', 
	'5 speed manual', 
	'5 speed manual transmission', 
	'5 speed+1(r)', 
	'5 speed,5 forward, 1 reverse', 
	'5-speed', 
	'5-speed`', 
	'five speed', 
	'five speed manual', 
	'five speed manual transmission', 
	'five speed manual transmission gearbox', 
]
gear_box_mapping['6 speed'] = [
	'6', 
	'6 speed', 
	'6 speed at', 
	'6 speed automatic', 
	'6 speed geartronic', 
	'6 speed imt', 
	'6 speed ivt', 
	'6 speed mt', 
	'6 speed with sequential shift', 
	'6-speed', 
	'6-speed at', 
	'6-speed automatic', 
	'6-speed autoshift', 
	'6-speed cvt', 
	'6-speed dct', 
	'6-speed imt', 
	'6-speed ivt', 
	'6-speed`', 
	'six speed  gearbox', 
	'six speed automatic gearbox', 
	'six speed automatic transmission', 
	'six speed geartronic, six speed automati', 
	'six speed manual', 
	'six speed manual transmission', 
	'six speed manual with paddle shifter', 
]
gear_box_mapping['7 speed'] = [
	'7 speed', 
	'7 speed 7g-dct', 
	'7 speed 9g-tronic automatic', 
	'7 speed cvt', 
	'7 speed dct', 
	'7 speed dsg', 
	'7 speed dual clutch transmission', 
	'7 speed s tronic', 
	'7-speed', 
	'7-speed dct', 
	'7-speed dsg', 
	'7-speed pdk', 
	'7-speed s tronic', 
	'7-speed s-tronic', 
	'7-speed steptronic', 
	'7-speed stronic', 
	'7g dct 7-speed dual clutch transmission', 
	'7g-dct', 
	'7g-tronic automatic transmission',
	'amg 7-speed dct',	
	'mercedes benz 7 speed automatic',
]
gear_box_mapping['8 speed'] = [
	'8', 
	'8 speed', 
	'8 speed cvt', 
	'8 speed multitronic', 
	'8 speed sport', 
	'8 speed tip tronic s', 
	'8 speed tiptronic', 
	'8-speed', 
	'8-speed automatic', 
	'8-speed automatic transmission', 
	'8-speed dct', 
	'8-speed steptronic', 
	'8-speed steptronic sport automatic transmission', 
	'8-speed tiptronic', 
	'8speed',
	'amg speedshift dct 8g', 
]
gear_box_mapping['9 speed'] = [
	'9 -speed', 
	'9 speed', 
	'9 speed tronic', 
	'9-speed', 
	'9-speed automatic', 
	'9g tronic', 
	'9g-tronic', 
	'9g-tronic automatic', 
	'amg speedshift 9g tct automatic',
]
gear_box_mapping['10 speed'] = [
	'10 speed', 
]
gear_box_mapping['cvt'] = [
'cvt', 
'e-cvt', 
'ecvt', 
]
gear_box_mapping['direct drive'] = [
'direct drive', 
]
gear_box_mapping['fully automatic'] = [
	'automatic transmission', 
	'fully automatic',
]
gear_box_mapping['nan'] = [
'nan',
'ags', 
'imt', 
'ivt', 
]

mapping_dict = {v: k for k, lst in gear_box_mapping.items() for v in lst}
cars_df['Gear Box'] = cars_df['Gear Box'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed            25203
6 speed             7459
7 speed             1842
8 speed             1150
4 speed              760
cvt                  474
nan                  471
9 speed              399
fully automatic       20
1 speed               15
direct drive          11
10 speed               9
Name: Gear Box, dtype: int64

--- 

##### `Drive Type`

In [23]:
cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd                                         27456
nan                                          4496
rwd                                          2248
awd                                          1082
2wd                                           648
4wd                                           570
2 wd                                          369
4x2                                           297
4x4                                           229
front wheel drive                             176
two wheel drive                                98
all wheel drive                                32
rear wheel drive with esp                      29
two whhel drive                                26
permanent all-wheel drive quattro              21
rwd(with mtt)                                  14
rear-wheel drive with esp                       7
4 wd                                            7
all-wheel drive with electronic traction        5
four whell drive                                2


We will once again take the same approach and define a mapping since many categories are redundant here.

In [24]:
drive_type_mapping = {}
drive_type_mapping['fwd'] = ['fwd', 'front wheel drive']
drive_type_mapping['2wd'] = ['2wd', 'two wheel drive', '2 wd', 'two whhel drive']
drive_type_mapping['rwd'] = ['rwd', 'rear wheel drive with esp', 'rear-wheel drive with esp', 'rwd(with mtt)']
drive_type_mapping['awd'] = ['awd', 'all wheel drive', 'all-wheel drive with electronic traction', 'permanent all-wheel drive quattro']
drive_type_mapping['4wd'] = ['4wd', '4 wd', '4x4', 'four whell drive']
drive_type_mapping['nan'] = ['nan', '3']

mapping_dict = {v: k for k, lst in drive_type_mapping.items() for v in lst}
cars_df['Drive Type'] = cars_df['Drive Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd    27632
nan     4497
rwd     2298
2wd     1141
awd     1140
4wd      808
4x2      297
Name: Drive Type, dtype: int64

---

##### `Steering Type`

In [25]:
# Steering Type
cars_df[columns_unexpected_cardinality[4]].value_counts()

power         31920
electric       3895
nan             808
manual          652
electronic      344
electrical      138
epas             49
hydraulic         5
mt                1
motor             1
Name: Steering Type, dtype: int64

There are 2 kinds of steering types (broadly) - 
1. Power (electric)
2. Manual (hydraulic, etc.)

In [26]:
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electrical', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electric', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electronic', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('epas', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('mt', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('motor', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')

cars_df[columns_unexpected_cardinality[4]].value_counts()

power     36348
nan         808
manual      657
Name: Steering Type, dtype: int64

---

##### `Front Brake Type` and `Rear Brake Type`

In [27]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc                                        22642
ventilated disc                             13178
ventilated discs                              400
solid disc                                    398
nan                                           327
disc & caliper type                           296
disk                                          205
ventilated disk                                78
drum                                           67
multilateral disc                              42
vantilated disc                                39
disc, 236 mm                                   22
ventlated disc                                 17
vacuum assisted hydraulic dual circuit w       12
ventillated disc                               11
disc & drum                                    10
disc,internally ventilated                      9
electric parking brake                          8
264mm ventilated discs                          8
caliper ventilated disc                         6


In [28]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum                                        29260
disc                                         5272
ventilated disc                               757
solid disc                                    612
nan                                           326
self-adjusting drum                           322
discs                                         301
disc & caliper type                           296
ventilated discs                              178
leading-trailing drum                         134
leading & trailing drum                        75
ventilated drum                                46
drums                                          40
self adjusting drum                            33
disc & drum                                    32
self adjusting drums                           31
drums 180 mm                                   22
drum in discs                                  17
vacuum assisted hydraulic dual circuit w       12
262mm disc & drum combination                   8


In [29]:
brake_type_mapping = {}
brake_type_mapping['disc'] = [
	'disc',
	'260mm discs',
	'disc brakes',
	'disc, 236 mm',
	'discs',
	'disk',
	'multilateral disc',
	'solid disc',
	'electric parking brake',
	'abs',
]
brake_type_mapping['ventilated disc'] = [
	'264mm ventilated discs',
	'booster assisted ventilated disc',
	'caliper ventilated disc',
	'disc brakes with inner cooling',
	'disc,internally ventilated',
	'vantilated disc',
	'ventilated & grooved steel discs',
	'ventilated disc',
	'ventilated disc with twin pot caliper',
	'ventilated discs',
	'ventilated disk',
	'ventillated disc',
	'ventillated discs',
	'ventlated disc',
	'ventilated drum in discs',
	'ventialte disc',
	'ventialted disc',
]
brake_type_mapping['carbon ceramic'] = [
	'carbon ceramic brakes',
	'carbon ceramic brakes.',
]
brake_type_mapping['disc & drum'] = [
	'disc & drum',
	'228.6 mm dia, drums on rear wheels',
	'262mm disc & drum combination',
	'drum in disc',
	'drum in discs',
]
brake_type_mapping['drum'] = [
	'drum',
	'203mm drums',
	'drum`',
	'drums',
	'drums 180 mm',
	'booster assisted drum',
	'drum brakes',
	'leading & trailing drum',
	'leading-trailing drum',
	'self adjusting drum',
	'self adjusting drums',
	'self-adjusting drum',
	'single piston sliding fist',
	'ventilated drum',
	'tandem master cylinder with servo assist',

]
brake_type_mapping['caliper'] = [
	'six piston claipers',
	'twin piston sliding fist caliper',
	'vacuum assisted hydraulic dual circuit w',
	'four piston calipers',
	'disc & caliper type',
]

In [30]:
mapping_dict = {v: k for k, lst in brake_type_mapping.items() for v in lst}
cars_df['Front Brake Type'] = cars_df['Front Brake Type'].replace(mapping_dict)
cars_df['Rear Brake Type'] = cars_df['Rear Brake Type'].replace(mapping_dict)

In [31]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc               23334
ventilated disc    13759
nan                  327
caliper              312
drum                  70
disc & drum           10
carbon ceramic         1
Name: Front Brake Type, dtype: int64

In [32]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum               29973
disc                6195
ventilated disc      944
nan                  326
caliper              310
disc & drum           64
carbon ceramic         1
Name: Rear Brake Type, dtype: int64

---

##### `Tyre Type`

In [33]:
cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless,radial               21770
tubeless                       5723
tubeless, radial               4994
tubeless tyres                 2408
radial                          606
radial, tubeless                389
tubeless radial tyres           301
radial, tubless                 296
nan                             257
tubeless tyres, radial          224
tubeless,radials                212
radial,tubeless                 158
tubless, radial                  92
runflat                          69
tubeless tyre                    50
tubeless, runflat                48
run-flat                         47
runflat tyres                    41
radial tubeless                  33
tubeless,runflat                 31
tubeless radial                  17
runflat tyre                     16
tubeless, radials                 7
radial with tube                  4
tubeless. runflat                 3
tubeless radial tyrees            3
tubeless tyres mud terrain        3
radial tyres                

In [34]:
tyre_type_mapping = {}
tyre_type_mapping['tubeless'] = [
	'tubeless tyres',
	'tubeless',
	'tubeless tyres mud terrain',
	'tubeless tyre',
]
tyre_type_mapping['tubeless radial'] = [
	'tubeless, radial',
	'tubeless,radial',
	'tubeless tyres, radial',
	'tubeless radial tyres',
	'radial, tubeless',
	'radial',
	'tubless, radial',
	'radial tubeless',
	'tubeless radial',
	'tubeless,radials',
	'tubeless radials',
	'radial,tubeless',
	'tubeless radial tyre',
	'radial, tubless',
	'tubless radial tyrees',
	'tubeless , radial',
	'tubeless, radials',
	'radial tyres',
]
tyre_type_mapping['runflat'] = [
	'runflat tyres',
	'runflat',
	'tubeless,runflat',
	'run-flat',
	'runflat tyre',
	'tubeless, runflat',
	'tubeless. runflat',
	'tubeless.runflat',
	'tubeless radial tyrees',
]
tyre_type_mapping['tube'] = [
	'radial with tube',
]

In [35]:
mapping_dict = {v: k for k, lst in tyre_type_mapping.items() for v in lst}
cars_df['Tyre Type'] = cars_df['Tyre Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless radial    29108
tubeless            8184
runflat              260
nan                  257
tube                   4
Name: Tyre Type, dtype: int64

---

##### `Fuel Injection System`

Although we expect a high cardinality from this feature, it's still worth a look if we could reduce the number of catgories by losing some of the variance.

(This step is optional and can be skipped. Also the mapping that is done is approximate and to the best of my knowledge, your mapping could differ)

In [36]:
cars_df['Fuel Suppy System'].value_counts()

mpfi                                      13443
crdi                                       9165
nan                                        5502
direct injection                           3831
pgm-fi                                      882
                                          ...  
intake port(multi-point)                      1
mfi                                           1
distributor-type diesel fuel injection        1
tfsi                                          1
pgm-fi (programmed fuel inject                1
Name: Fuel Suppy System, Length: 71, dtype: int64

In [37]:
fuel_injection_mapping = {
    "Gasoline Port Injection": [
    	"intelligent-gas port injection", 
    	"i-gpi",
    	"dohc",
    	"pfi"
	],
    "Multi-Point Fuel Injection": [
    	"mpfi", 
    	"multi-point injection", 
    	"mpfi+lpg", 
    	"mpfi+cng", 
    	"multipoint injection", 
    	"smpi", 
    	"mpi",
    	"multi point fuel injection",
    	"dpfi",
    	"mfi",
    	"multi point injection",
    	"msds",
    	"cng"
    ],
    "Electronic Fuel Injection": [
	    "efi(electronic fuel injection)", 
	    "efi", 
	    "efi (electronic fuel injection)", 
	    "efic", 
	    "electronic fuel injection", 
	    "electronically controlled injection", 
	    "electronic injection system", 
	    "sefi",
	    "egis",
	    "efi (electronic fuel injection",
	    "efi",
        "efi -electronic fuel injection",
    ],
    "Direct Injection": [
    	"direct injection", 
    	"direct injectio", 
    	"direct fuel injection",
    	"direct engine",
    ],
    "Common Rail Injection": [
    	"crdi", 
    	"common rail", 
    	"common rail injection", 
    	"common rail direct injection", 
    	"common rail direct injection (dci)", 
    	"common-rail type", 
    	"advanced common rail", 
    	"common rail system", 
    	"common rail diesel", 
    	"pgm-fi (programmed fuel injection)", 
    	"pgm-fi (programmed fuel inje", 
    	"pgm - fi", 
    	"pgm-fi", 
    	"pgm-fi (programmed fuel inject",
    	"direct injection common rail",
    	"cdi"
    ],
    "Distributor-Type Fuel Injection": [
    	"dedst", 
    	"distribution type fuel injection", 
    	"distributor-type diesel fuel injection",
    ],
    "Indirect Injection": [
    	"indirect injection",
    	"idi"
    ],
    "Gasoline Direct Injection": [
    	"gdi", 
    	"gasoline direct injection",
    	"tfsi",
    	"tsi",
    	"tgdi"
    ],
    "Turbo Intercooled Diesel": [
    	"tcdi", 
    	"turbo intercooled diesel",
    	"tdci"
    ],
    "Intake Port Injection": [
    	"intake port(multi-point)"
    ],
    "Diesel Direct Injection": [
    	"ddi", 
    	"ddis"
    ],
    "Variable Valve Timing Injection": [
    	"dual vvt-i", 
    	"vvt-ie", 
    	"ti-vct"
    ],
    "Three-Phase AC Induction Motors": [
    	"3 phase ac induction motors"
    ],
    "Electric": [
    	"electric", 
    	"isg"
    ],
}

mapping_dict = {v: k for k, lst in fuel_injection_mapping.items() for v in lst}
cars_df['Fuel Suppy System'] = cars_df['Fuel Suppy System'].replace(mapping_dict)

cars_df['Fuel Suppy System'].value_counts()

Multi-Point Fuel Injection         14055
Common Rail Injection              11822
nan                                 5502
Direct Injection                    3837
Electronic Fuel Injection           1593
Gasoline Direct Injection            653
Gasoline Port Injection              100
Diesel Direct Injection               99
Turbo Intercooled Diesel              89
Indirect Injection                    21
Variable Valve Timing Injection       15
Three-Phase AC Induction Motors       11
Distributor-Type Fuel Injection        8
Electric                               7
Intake Port Injection                  1
Name: Fuel Suppy System, dtype: int64

##### With this column, all of the features suspected of high cardinality are handled. We can move on to the other things that were pointed out earlier in the code.

---

#### Converting features to numerical (and/or boolean)

The features we wish to convert are the following:
1. `pu` to float
2. `Max Power` to 2 separate columns: `Max Power Delivered` and `Max Power At`
3. ~~`No of Cylinder` to int (from float)~~
4. ~~`Values per Cylinder` to int (from float)~~
5. `BoreX Stroke` to 2 separate columns: `Bore` and `Stroke`
6. `Turbo Charger` to boolean
7. `Super Charger` to boolean
8. `Length` to int 
9. `Width` to int
10. `Height` to int
11. `Wheel Base` to int
12. `Front Tread` to int
13. `Rear Tread` to int
14. `Kerb Weight` to int
15. `Gross Weight` to int
16. ~~`Seating Capacity` to int (from float)~~
17. `Turning Radius` to float
18. `Top Speed` to int
19. `Acceleration` to float
20. ~~`No Door Numbers` to int (from float)~~
21. `Cargo Volumn` to int
22. `Compression Ratio` to float
23. `Alloy Wheel Size` to float
24. `Ground Clearance Unladen` to int

In [38]:
# Make a copy of the data, cars_df will now serve as a checkpoint
cars_df2 = cars_df.copy()

# Sanity check
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,180-liters,used,uttar pradesh,silver,first,nan,NaN,nan,nan
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,235-litres,used,maharashtra,grey,first,Gasoline Port Injection,11.0:1,nan,nan
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,4.000,400-litres,used,delhi,silver,second,nan,NaN,nan,nan
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,4.000,nan,used,delhi,silver,second,Multi-Point Fuel Injection,NaN,13,nan
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,5.000,nan,used,maharashtra,white,first,nan,12.0+-.03,nan,nan


In [39]:
# A utility function to get a number from a string
def converst_to_number(x, conv: str = 'float'):
    x = str(x)
    new_str = ''
    is_dec = True
    for a in x:
        if 48 <= ord(a) <= 57:
            new_str += a
            continue
        elif a == ',' or a == '_':
            continue
        elif a == '.' and is_dec:
            new_str += a
            is_dec = False
        else:
            break
    
    if new_str == '':
        return None
    
    if conv == 'int':
        return int(new_str)
    
    return float(new_str)

def get_begin_number(x):
    return converst_to_number(x, 'int')

def get_begin_float(x):
    return converst_to_number(x, 'float')

# def get_begin_number(x):
#     x = str(x)
#     reg_match = re.search(r'\d+', x)
#     if reg_match is None:
#         return None
#     return int(reg_match.group())

# def get_begin_float(x):
#     x = str(x)
#     reg_match = re.search(r'\d+\.?\d+', x)
#     if reg_match is None:
#         return None
#     return float(reg_match.group())

---

#### `pu`

This is our target column (the price of our used cars). This should be a float or int.

In [40]:
cars_df2['pu'].sample(5)

usedCarSkuId
f26e7e5c-b0ad-42e3-a824-e0b9c9981814     1,64,299
9343a8fe-9c41-4079-8615-44b3aa2f1443     2,50,000
c6fe217c-19fb-48e0-bdbd-164516212d81       98,414
fdc85bdd-2991-43a1-9ec1-505fbb4216c2    10,97,000
a51b6fe3-3117-4d5e-9edf-7b860a0ff0f4     1,35,000
Name: pu, dtype: object

In [41]:
# We simply need to replace the ',' to with '' and convert
cars_df2['pu'] = cars_df2['pu'].str.replace(',', '').astype(float)

cars_df2['pu'].sample(5)

usedCarSkuId
6c1fd6d2-65ea-4f45-bd3e-8271a6f9d7ea   759000.000
a92a4067-62f0-4dbe-b54e-0ba6842fc7f2   135000.000
87570dad-fa42-4b06-86d5-036476044a55   293000.000
0b8aa648-58cf-4c4e-b430-9fadbdf19e3c   310000.000
af1beace-ceef-4d1f-90cb-20e24c2ccd5e   275000.000
Name: pu, dtype: float64

---

#### `Max Power` and `Max Torque`

We will be separating this column into 2 columns: `Max Power Delivered` and `Max Power At` (containing the rpm at which the max power is reach)
Same with `Max Torque`

In [42]:
cars_df2[['Max Power', 'Max Torque']].sample(5)

,Max Power,Max Torque
usedCarSkuId,,
d9eaa650-d9a3-4dfc-a10e-b4154beb49ff,"65@4,000 (ps@rpm)","16@2,000 (kgm@rpm)"
96da63c1-cdd7-40a8-a84f-6c23383df688,68.05bhp@5500rpm,99.04nm@4500rpm
45b7c038-4354-456f-bc35-acde91fcd0a1,58.16bhp@6200rpm,77nm@3500rpm
b75efbc9-ae50-41d8-a718-9700c07f9d40,88.76bhp@4000rpm,219.6nm@1500-2750rpm
1a241897-f6df-4c04-87d2-d3c250c0ca96,88.8bhp@4000rpm,200nm@1750rpm


In [43]:
c = cars_df2.copy()
cars_df2['Max Power Delivered'] = cars_df2['Max Power'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Power At'] = cars_df2['Max Power'].str.split('@').str[1].apply(get_begin_float).astype(float)


def get_rpm_average(x):
    x = str(x)
    if '-' in x:
        p1 = get_begin_float(x.split('-')[0])
        p2 = get_begin_float(x.split('-')[1])
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        
        return (p1 + p2)/2
    else:
        return get_begin_float(x)

cars_df2['Max Torque Delivered'] = cars_df2['Max Torque'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Torque At'] = cars_df2['Max Torque'].str.split('@').str[1].apply(get_rpm_average).astype(float)

In [44]:
cars_df2[['Max Power Delivered', 'Max Power At', 'Max Torque Delivered', 'Max Torque At']].sample(5)

,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,
e5af6476-aad4-4f68-b3dd-e4b253ca19ee,126.320,4000.000,259.800,2325.000
ea3df4a2-5247-439e-93ee-921cdaac8169,88.500,4000.000,200.000,1750.000
06d200b8-b937-4839-9218-14c97cfd6516,67.050,5500.000,90.000,3500.000
fc54d510-3451-4f79-a478-57adb067feff,171.500,3400.000,360.000,2300.000
bcc743bf-d6bb-46b0-a2fb-a72185f9680e,126.320,4000.000,259.800,2325.000


Now we can safely drop `Max Power` from our dataframe

In [45]:
cars_df2.drop(columns=['Max Power'], inplace=True, axis=1)
cars_df2.drop(columns=['Max Torque'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,silver,first,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,grey,first,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,silver,second,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,silver,second,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,white,first,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000


---

#### `BoreX Stroke`

We will make 2 new columns `Bore` and `Stroke` and remove the original column.


In [46]:
cars_df2['BoreX Stroke'].sample(10)

usedCarSkuId
1ce7d5be-fddc-4e5d-9f13-052fed4b822a    74.0 x 75.5 mm
ac25ccf8-5422-43b7-bb93-d5c9647db3a4               nan
3295cd5c-80cd-437e-bd48-413151345f70      76 x 80.5 mm
6010b3cc-0cb1-447d-b45b-6e98d04883d5               nan
aceecc27-2507-4e92-8add-393591bc9444               nan
8172a658-3298-476c-b74d-c099955c99c7               nan
673a7c49-510f-4890-b16e-b31fcb9afa12               nan
6e114505-e768-4f51-8876-442c2a98967b        69 x 72 mm
bf6684b2-c1ef-460e-a32a-c1a0140e679f               nan
a647483e-773b-4cf6-9415-95627fdff037               nan
Name: BoreX Stroke, dtype: object

In [47]:
cars_df2['Bore'] = cars_df2['BoreX Stroke'].str.split('x').str[0].apply(get_begin_float).astype(float)
cars_df2['Stroke'] = cars_df2['BoreX Stroke'].str.split('x').str[1].apply(get_begin_float).astype(float)

In [48]:
cars_df2[['Bore', 'Stroke']].sample(10)

,Bore,Stroke
usedCarSkuId,,
2de9fa3e-0ab6-429b-bef3-ce15c2aadb79,NaN,NaN
03214065-ce32-4355-bee7-e841e20a788f,71.000,NaN
16c79667-8021-47e2-909c-b7beee3f417f,NaN,NaN
392fd536-517e-41ce-b61c-91fd31e8f194,NaN,NaN
2679de2f-eb43-4e11-b274-caeaa22eea40,73.000,NaN
2c6e650f-835c-47af-964b-13c12e713557,77.200,NaN
d3e876b8-5ccf-40a4-b4ca-41c0a8de5659,NaN,NaN
8d6495d7-3f80-45d2-81f9-c04d31c3ee5f,NaN,NaN
d6e0a444-8c68-424d-90a4-3996dcee5985,NaN,NaN


In [49]:
cars_df2.drop(columns=['BoreX Stroke'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000,69.000,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000,73.000,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000,NaN,NaN


---

#### `Turbo Charger` and `Super Charger`

Convert them to boolean

In [50]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
b8c37660-d8ac-4993-a6f8-555a50286530,nan,nan
57c066b4-045f-4cbd-bd1e-b51f2706186d,yes,no
0d7eb231-4ff9-4b0e-b161-ea7bb440c7bf,no,no
1581b4a5-0482-4f45-9c69-17b8c813e01a,no,no
a9d64673-66e8-4933-a234-f18b92b6d218,nan,nan


In [51]:
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

In [52]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
3249a264-f445-480e-89c5-cbf9675b4939,False,False
36a6cd86-1356-4ce6-ab7d-7071d9997ca5,True,True
18191309-3bdb-4664-8e55-63dc2adea0d8,False,False
7bdee52d-6fe2-4c9e-9f74-388f156ef1b9,False,False
a5f36e4e-7dbe-4e66-8eba-69bef042ac99,True,True


---

#### `Length`, `Width`, `Height`, `Wheel Base`

Convert these columns to int

In [53]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(10)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
a92170ff-44df-43f1-ab55-e8d6ce4a997c,nan,nan,nan,nan
4e3a8c96-7402-4e49-8778-24613ca1effb,3995mm,1790mm,1640mm,2500mm
a4cbd1f4-0a2e-4109-b57a-acfc46dc2044,4925mm,1700mm,1825mm,3150mm
08a0ca54-9281-424f-aed2-ca14369ce27c,4370mm,1700mm,1475mm,2570mm
cc77affe-beab-4f69-b370-e243dd2bdba4,3801mm,1665mm,1530mm,2450mm
580f972e-09f4-4595-b8cb-e701d47e1987,3985mm,1734mm,1505mm,2570mm
35e2b956-8ed3-4f1f-a74d-0bfabaf72a4d,3994mm,1811mm,1607mm,2498mm
0d5f8574-5fab-4b85-9681-82a967de659a,3599mm,1495mm,1700mm,2400mm
3917f629-adec-43e2-a4ac-a9b0336c337a,4630mm,1777 mm,1432mm,2699mm


In [54]:
for col in ['Length', 'Width', 'Height', 'Wheel Base']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [55]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(5)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
de256d12-2859-4f6e-97cb-fc37839f0699,4545.000,1750.000,1450.000,2700.000
0b7d56e3-bfbe-41b4-811d-bb25c72c839a,4620.000,1776.000,1475.000,2700.000
a0364ac0-971d-4f6c-b6b9-58dff3a6f7b6,3731.000,1579.000,1474.000,NaN
32fb4a35-c600-4dd1-b0ba-0180446d92b0,4370.000,1700.000,1475.000,2570.000
d64283b6-59c1-464d-b919-b658a380b47b,4270.000,1780.000,1630.000,2590.000


---

#### `Front Tread` and `Rear Tread`

In [56]:
cars_df2[['Front Tread', 'Rear Tread']].sample(10)

,Front Tread,Rear Tread
usedCarSkuId,,
80302c9a-5f70-4767-967c-1e6fd7f15cf7,nan,nan
16bd3c60-6f48-4cfd-b36c-4da2d100b409,nan,nan
f9284d8f-ec87-4671-8591-2f9a23411c63,1479mm,1493mm
ce71dbb8-10e5-4028-a2ab-78886abeae21,"1,505 mm","1,503 mm"
e678e73d-3c76-4d81-82a8-631eaa21bdc2,1485mm,1495mm
cebaf820-0d49-4606-a3d8-e2eab3138cbd,nan,nan
1d05b213-6d6f-4c9d-8a25-e3f0667b3142,1495mm,1505mm
ac35fc2b-7e68-4936-b325-a4d023d28b3a,1475mm,1465mm
89c67788-27e4-4113-beb1-bcd71e5048e4,1470mm,1480mm


In [57]:
for col in ['Front Tread', 'Rear Tread']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [58]:
cars_df2[['Front Tread', 'Rear Tread']].sample(5)

,Front Tread,Rear Tread
usedCarSkuId,,
5201d23f-fa0e-4055-a7e8-5ac8b1611cdd,1386.000,1368.000
ec8be169-0f9e-4687-84eb-cf5b46f8d071,1495.000,1505.000
1967609e-5086-48e9-8ff5-b1253457a80b,1480.000,1465.000
5c89465a-5c22-47aa-87bc-5e1cc945ca87,1485.000,1495.000
b99d21fd-9fe1-4e3e-ae12-df0330aea5a3,1400.000,1385.000


---

#### `Kerb Weight` and `Gross Weight` 

Convert weights to numerical features

In [59]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
b8dffd90-9217-4578-805c-38d4f169d35d,nan,nan
6ea6d288-b0f0-47b1-a40d-6415420952b5,1030kg,nan
08d301cf-b63e-457a-96a7-5595ecbec007,1641 kgs,nan
607e3330-a82c-4a58-8f4f-704a85539c5a,960kg,1405kg
6269f2ac-5ba1-4704-844e-60a32cffff41,1161kg,nan
6f5dc61d-f743-4b9e-b1bb-aed83d351f62,1329kg,1640kg
481d91b2-1278-4acc-b31d-3689795ecfab,nan,nan
7ed3e762-a245-4f31-97b8-ab9e674bc778,1061kg,1436kg
8fb127a6-9d95-456a-a6e0-288d2fb7ec8f,1675kg,2220kg


In [60]:
for col in ['Kerb Weight', 'Gross Weight']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [61]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
aa5fd2af-8646-4a03-9528-3b3c21305043,1879.000,NaN
0e4e09f2-ba58-4cc1-8779-80f3763a7c9c,NaN,NaN
d00d856c-7d97-4a69-99a3-1fe8c0e12ff1,NaN,NaN
2e45d982-4bdf-4d13-b273-4ac751b1bb54,NaN,NaN
f96dcf5c-efb6-4a60-b411-862d3cdd156b,NaN,NaN
fdccd770-860c-4419-8036-5624c6043556,NaN,NaN
d10ade31-8ffb-442d-84ff-d0629e70216c,1055.000,1505.000
1b0f78bd-1c81-4e2f-ab12-2269943d2c33,885.000,1350.000
cf7f63ee-e6e0-407f-81ac-edacb1d7b93f,1562.000,NaN


---

#### `Turning Radius`,

In [62]:
cars_df2['Turning Radius'].sample(10)

usedCarSkuId
23167941-b330-4093-b900-0f21073f9f09    4.7 metres
d18d9ee3-0bfe-49eb-8e8e-10cd43902bd9    4.7 metres
8515382f-c6d0-43e1-b4b5-9f5aedc7c615    4.6 metres
d2276422-3412-408a-90e9-dba2c8bf9451    5.5 meters
d9ccea00-9fcc-423f-8739-b037faf9c92e           nan
8b0b58e5-2e4b-40e4-889c-17b05230e6e0         5.3 m
7a54e19a-c3fa-4b8d-b92c-3a77053a0116           5.3
4ad7e9d3-f0af-4639-8820-cdedc2061bed    4.8 metres
5e693a1e-ea1a-4e4a-9711-4262be7e91af           nan
8d4a5b2f-9bea-4eee-80c1-3ebb4c7069d6           nan
Name: Turning Radius, dtype: object

In [63]:
cars_df2['Turning Radius'] = cars_df2['Turning Radius'].apply(get_begin_float).astype(float)

In [64]:
cars_df2['Turning Radius'].sample(5)

usedCarSkuId
feb3f66a-750f-4c2f-9e2d-310bb6ce5d53   4.600
d1db18c2-8d5c-48fa-96e2-9625f7606d44   4.600
9b9012be-6968-42e6-a8ef-d4c6fe4495ea   4.600
89679497-32ac-4db0-a25b-c85aca412b3e   4.400
99a6cbcf-10f3-46dc-9491-f68b1067b2d8   5.100
Name: Turning Radius, dtype: float64

---

#### `Top Speed` and `Acceleration`

In [65]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
f29534c2-95dc-4185-a684-89ab0da4d831,nan,17.7
5cf54683-4414-4b9e-91cf-d4a67804c231,200 kmph,10.1 seconds
8d0ac584-6fb2-4c7b-8a3d-78cd5760434f,nan,nan
9dae0876-d3da-4553-ac32-c4cf20f539a9,151.63 kmph,13.21 seconds
a779e910-5a14-4809-8481-917bf00a380c,186km/hr,nan
b118f959-b05b-4aec-9eda-7aed95f9998f,160 kmph,15 seconds
ed743820-789f-40ec-a381-8faccf04232f,200 kmph,10.1 seconds
4e8cef51-d25e-405c-aec1-7b89100db93a,nan,nan
253e5292-9e82-4cdb-a517-93109b595fcd,nan,nan


In [66]:
cars_df2['Top Speed'] = cars_df2['Top Speed'].apply(get_begin_float).astype(float)
cars_df2['Acceleration'] = cars_df2['Acceleration'].apply(get_begin_float).astype(float)

In [67]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
babafaf2-e5bf-4e01-9afb-c280b866f09e,NaN,12.600
2222b43b-7922-4c4f-900f-9dc7fb072eb4,148.000,14.800
82f79801-f348-4fd5-96b9-3d1ee1164c8c,172.000,13.700
bdeab2ce-3605-4750-8d28-7ec878a3e53f,NaN,NaN
41c852bd-f5c3-40c5-9859-9e99402332b7,NaN,NaN
d61d29ef-3645-4db1-9cf0-f3fd5e63253b,195.000,10.000
a59985ac-7aee-4243-8da9-dde7f4dcfad1,NaN,NaN
3444b704-ded7-4bc7-90ee-7217d9d9a667,NaN,10.030
392fd536-517e-41ce-b61c-91fd31e8f194,135.000,19.000


---

#### `Cargo Volumn` and `Ground Clearance Unladen`

Convert these columns to int

In [68]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
ef1abf32-b502-4e7e-8fba-70ab2ae97709,nan,nan
34bd5fa3-da7d-44f4-87bd-83419b8313d1,433,nan
36b2c3ad-4609-4a76-a169-0f4f2b1042b4,nan,nan
9e64f708-7845-4653-be89-e5ba5bf519bb,475-litre,nan
145bbd59-1cc7-483b-a8cf-9fbeaa96a6d7,223-litres,nan
35c31acd-cd20-480f-929e-9d60c2bab9bf,400-litres,nan
f264abae-e6d2-4ffc-9d37-952df672b23e,236-liters,nan
42c0ce21-c74f-4b6a-8407-d41e2c872633,425,nan
daa197cf-cba5-493d-ab8f-484470fb7452,175-litres,nan


In [69]:
for col in ['Cargo Volumn', 'Ground Clearance Unladen']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [70]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
efb37b36-c0e8-4619-9895-1c9c6d0ae19e,350.000,190.000
615524ac-c484-4a1d-a068-894fe71d6d13,460.000,NaN
464c03f7-9087-41c9-ad54-4426e6036c6a,480.000,NaN
6cf432f0-8188-4bc2-a6bb-39399e7e4b20,510.000,NaN
2c6d559b-dcef-4fba-8d15-9d029b70ae71,275.000,NaN
50ec9936-abc4-484b-80a2-6843b687aeaf,475.000,NaN
f4870ce0-150c-4c22-8eb4-19ec9d4dc3fa,222.000,185.000
c92688c1-e28e-46cc-8da1-e28418e20e1d,155.000,192.000
36b782da-492d-489a-b087-bd32ee61c9d6,336.000,NaN


---

#### `Compression Ratio` and `Alloy Wheel Size`

These columns should be converted to floats

In [71]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
f2bf9aa0-e82e-4e96-badf-b8b1ddbdcfc5,15,10.1:1
1e63a2f2-beb9-4263-bf32-4b033bf55dd7,nan,NaN
f20ac7cd-3aa9-4025-8109-202e887410f4,14,NaN
615fbd73-f7f0-4f11-b1ba-e7041106988b,16,NaN
0e002f53-b2b5-4695-b8c4-27a263bb2d6c,15,16.5:1
4802e78b-3ae5-4df2-bb22-d1ebc4cf32d6,16,NaN
c4de34d3-e418-471e-80bb-3a93355ac53e,18,NaN
548de5a7-98cb-4063-bda4-9e6388ce8617,14,NaN
68830472-7586-4039-aaf8-ce5d8e5af095,nan,NaN


In [72]:
for col in ['Alloy Wheel Size', 'Compression Ratio']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [73]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
68ca3291-f62f-439c-8334-b7d94da8627f,NaN,NaN
1b9c879d-b928-4f9d-bdb2-11ae5b86e756,16.000,NaN
f3a45aa9-4142-48d9-b719-fdfac7deb0be,NaN,NaN
3fed42b5-4954-4b55-a371-8a1be8c9e52e,14.000,NaN
105463cc-9ca6-44b7-aa87-ccfc5652a204,16.000,16.500
2724028f-40eb-4a45-8354-321fb2a8e503,16.000,NaN
74d9a080-5db7-417d-b8e5-3d7d14c6d004,NaN,NaN
81c61932-5495-4407-a2ab-bc0873b01c41,15.000,NaN
454a7037-00d8-4c33-81a0-0f25d410050a,NaN,11.000


---
### `km`

The columns need to be converted to int64.

In [74]:
cars_df2['km'].sample(10)

usedCarSkuId
1f1635ac-df15-431e-850e-bd8dc77a8293      76,376
1db2ee60-6684-4126-a6f8-5fb29756628d      75,826
65afd2bc-0c24-428d-8fcb-fb00c5c3ead0       8,640
77d83904-2213-4d48-858b-b8122ba540d6    1,00,000
3e7bd697-de32-4f9f-a8e8-95d019f8001d      96,409
9cfed958-b5f3-464e-881e-336dfaa7102f      73,612
387b085f-5799-4678-b2ee-31fd062076af      60,984
d19a4f2c-4028-451a-9c90-92161a120e5f      59,613
5893c646-b9f9-4210-959b-56ad49a3fa43    1,20,000
166dec40-5f89-4728-a6cc-cbedb50fea0b      47,560
Name: km, dtype: object

In [75]:
cars_df2['km'] = cars_df2['km'].apply(get_begin_float).astype(float)

In [76]:
cars_df2['km'].sample(10)

usedCarSkuId
61046461-1423-47db-bd44-683924a176cb    50000.000
64e4513e-55dc-48fa-a599-857deb041be7    56824.000
a4be9d07-ece1-4de1-85f9-09f9e76d48d9    80000.000
a544a029-d324-46c9-8be8-caf1e42bb09d    22245.000
d4b98c15-5020-4625-b2fb-0895d89f0084    54200.000
f451737a-d711-46f3-8159-dbc937ba392f    51473.000
dd899639-439a-4c45-a1e3-16db340e070c    80000.000
1f1d9705-d669-4a92-8148-b5514eba2da3   102653.000
218cc69f-878c-4603-9f03-686bad54cf69    29908.000
4b325bb1-a291-4baf-a123-6e6a4438516b    34470.000
Name: km, dtype: float64

With this, we have converted all the columns to numerical that were noted.

In [77]:
cars_df2.dtypes.value_counts()

object     31
float64    27
int64       4
bool        3
dtype: int64

---

#### Fixing some column names

In [78]:
cars_df2 = cars_df2.rename(columns={
    'tt': 'transmission',
    'bt': 'body',
    'ft': 'fuel',
    'variantName': 'variant',
    'km': '',
    'pu': 'listed_price',
    'Values per Cylinder': 'Valves per Cylinder',
    'Value Configuration': 'Valve Configuration',
    'No Door Numbers': 'Doors',
    'Seating Capacity': 'Seats',
    'Cargo Volumn': 'Cargo Volume',
    'city_x': 'City',
})

In [79]:
cars_df2.head()

,loc,myear,body,transmission,fuel,,ip,images,imgCount,threesixty,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,NaN,NaN,58.160,6200.000,77.000,3500.000,69.000,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,Gasoline Port Injection,11.000,NaN,NaN,58.200,6000.000,78.000,3500.000,73.000,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,NaN,NaN,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.000,0,[{'img': ''}],0,False,...,Multi-Point Fuel Injection,NaN,13.000,NaN,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,nan,12.000,NaN,NaN,86.630,5500.000,121.500,4200.000,NaN,NaN


In [80]:
cars_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37813 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loc                       37813 non-null  object 
 1   myear                     37813 non-null  int64  
 2   body                      37813 non-null  object 
 3   transmission              37813 non-null  object 
 4   fuel                      37813 non-null  object 
 5                             37813 non-null  float64
 6   ip                        37813 non-null  int64  
 7   images                    37813 non-null  object 
 8   imgCount                  37813 non-null  int64  
 9   threesixty                37813 non-null  bool   
 10  dvn                       37813 non-null  object 
 11  oem                       37813 non-null  object 
 12  model                     37813 non-null  object 
 13  

#### With this, we are ready to move on to the next steps:
1. EDA
2. Feature Engineering and selection
3. Model Building and Optimization

In [81]:
import datetime

fileName = f"../data/clean/cleaned_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
# cars_df2.to_csv(filename)